In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from source.subtasks_datasets import *
from source.utils import *
from source.mb_classification import MusicBertClassifier
from source.mb_regression import MusicBertRegressor
from source.sound_transforms import *



transform = toMFCC()
dataset = ClassificationDatasetNumpy(source_file="data/GiantSteps/key_path.npy", transform=transform, use_cache=True)
train_dataloader, val_dataloader = split_and_load(dataset, workers=0, batch_size=16, split_size=0.75)

sound_features = dataset.features_size
num_classes = dataset.n_classes
evals = 0.5

print("Using %d features"%sound_features)

model = MusicBertClassifier(sound_features, num_classes, n_convs=4).cuda()
#model.BERT.load_pretrained()

Using 20 features


In [2]:
import mir_eval
import numpy as np

def key_eval (output, target):
    
    score_vector=[]
    
    for out, tar in zip(output, target):
        out_index = np.argmax(out)
        est_key = dataset.classes[out_index].replace(":", " ")
        ref_key = dataset.classes[tar.squeeze().long()].replace(":", " ")

        score = mir_eval.key.weighted_score(ref_key, est_key)
        score_vector.append(score)
        
    return np.mean(score_vector)

In [3]:
curve = model.train_model(train_dataloader, val_dataloader, epochs = 100, eval_per_epoch=evals, eval_fn=key_eval)

Accuracy (1799 steps) 0.1403


[0.14652777777777778,
 0.13784722222222223,
 0.14652777777777778,
 0.14027777777777778,
 0.15416666666666667,
 0.14027777777777778,
 0.16319444444444445,
 0.16736111111111113,
 0.17743055555555554,
 0.17777777777777778,
 0.17812499999999998,
 0.18055555555555555,
 0.18229166666666666,
 0.17847222222222225,
 0.1840277777777778,
 0.18125000000000002,
 0.19236111111111112,
 0.17118055555555556,
 0.16944444444444443,
 0.16597222222222222,
 0.14305555555555557,
 0.15625,
 0.19027777777777777,
 0.17256944444444444,
 0.17395833333333335,
 0.17326388888888888,
 0.15243055555555554,
 0.1795138888888889,
 0.16875,
 0.16493055555555555,
 0.18229166666666666,
 0.15243055555555554,
 0.15381944444444443,
 0.1857638888888889,
 0.18368055555555554,
 0.184375,
 0.17881944444444445,
 0.13923611111111112,
 0.17777777777777778,
 0.1684027777777778,
 0.16562500000000002,
 0.17916666666666664,
 0.1357638888888889,
 0.11562499999999999,
 0.12118055555555556,
 0.17083333333333334,
 0.14305555555555557,
 0.131

In [4]:
%matplotlib widget

plot_curve(model.validation_curve, evals, color="red")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model Accuracy: 14.03%


In [1]:
from source.subtasks_datasets import *
from source.utils import *
from source.mb_classification import MusicBertClassifier
from source.mb_regression import MusicBertRegressor
from source.sound_transforms import *

transform = toMFCC()
dataset = RegressionDatasetNumpy(source_file="data/GiantSteps/tempo_path.npy", transform=transform, use_cache=True)
train_dataloader, val_dataloader = split_and_load(dataset, workers=0, batch_size=16, split_size=0.75)

sound_features = dataset.features_size
output_size = dataset.output_size
evals = 0.5

print("Using %d features"%sound_features)

Instructions for updating:
non-resource variables are not supported in the long term
Using 20 features


In [2]:
model = MusicBertRegressor(sound_features, output_size, n_convs=4).cuda()
#model.BERT.load_pretrained()

In [5]:
import torch

def accuracy(output, target):
    
    avg, std = (136.6594954819277, 28.33103134639793)
    
    # we need to de-scale the values to compute accuracies
    output = std*output + avg
    target = std*target + avg
    
    # ACCURACY1
    delta = torch.abs(output-target)/target
    accuracy1 = (delta <= 0.04).float()
        
    #ACCURACY2
    multipliers = [1/3, 1/2, 1, 2, 3]
    accuracy2 = torch.zeros_like(accuracy1)
    
    for m in multipliers:
        current_target = target*m
        delta = torch.abs(output-current_target)/current_target
        local_acc = (delta <= 0.04).float()
        accuracy2 = accuracy2 + local_acc
    
    accuracy2 = accuracy2.bool().float()
    
    return torch.mean(accuracy1)#, torch.mean(accuracy2)


#model.evaluate(train_dataloader)#, eval_fn=accuracy)
curve = model.train_model(train_dataloader, val_dataloader, epochs = 50, eval_per_epoch=evals, eval_fn=accuracy)

1239 steps: validation=tensor(0.2456), training=tensor(0.3013)         


In [4]:
%matplotlib widget

plot_curve(model.validation_curve, evals, color="")
plot_curve(model.training_curve, evals, color="")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model Accuracy: 6.38%
Model Accuracy: 8.24%
